<h1>Sumário<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-e-Funções" data-toc-modified-id="Imports-e-Funções-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports e Funções</a></span></li><li><span><a href="#Download-dos-dados" data-toc-modified-id="Download-dos-dados-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><em>Download</em> dos dados</a></span></li><li><span><a href="#Lê-e-Ajusta-a-Tabela" data-toc-modified-id="Lê-e-Ajusta-a-Tabela-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Lê e Ajusta a Tabela</a></span></li><li><span><a href="#Escreve-a-Tabela" data-toc-modified-id="Escreve-a-Tabela-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Escreve a Tabela</a></span></li><li><span><a href="#Atualiza-Repositório-no-GitHub" data-toc-modified-id="Atualiza-Repositório-no-GitHub-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Atualiza Repositório no <em>GitHub</em></a></span></li></ul></div>

*Script* criado para organizar uma tabela com a estimativa da população de cada município do Brasil. Os dados foram obtidos com auxílio da [plataforma SIDRA](https://sidra.ibge.gov.br/pesquisa/EstimaPop/tabelas), obtendo os dados da [tabela 6579](https://sidra.ibge.gov.br/tabela/6579) e ajustando-os, elimintando cabeçalhos e rodapés etc.

# Imports e Funções

Inicialmente faz-se necessário importar os pacotes utilizados, bem como definir as funções que serão usadas posterirmente.

In [ ]:
%run '../codes/files/create_folders.py'

import os
import pandas as pd

create_folders('')

# *Download* dos dados

Deve-se definir a *url* que se localizam os dados, bem como o nome do arquivo a ser baixado. A *url* foi definida por meio do [modo de consulta do SIDRA](https://sidra.ibge.gov.br/tabela/6579), onde se definiu o *layout* da tabela e posterirmente acessando a função *Links de Compartilhar*.

O *layout* foi definido selecionando-se todos os anos disponíveis e os níveis territoriais foi selecionado apenas o nivel mais desagregado, que possibilita agregar os dados conforme o usuário quiser, ou seja, municípios.

Inicialmente está se trabalhando com o download de arquivos **_.csv_**, porém pretende-se futuramente tentar acessar os dados pela API do SIDRA (https://apisidra.ibge.gov.br/values/t/6579/n6/all/v/all/p/all)

![optionSIDRA](https://i.imgur.com/eCVDpEm.png) <!-- .element height="20%" width="20%" style="border: 0; background: None; box-shadow: None" -->

In [ ]:
filename = 'tab_EstimativaPopulação.csv'
urls = [('https://sidra.ibge.gov.br/geratabela?format=br.csv&name=tabela6579.csv&terr=NC&rank=-&query=t/6579/n6/all/v/all/p/all/l/v,p,t',
         filename)]

In [ ]:
%run '../codes/files/downloads.py'

download_urls(urls, 'data', GetFilenameFromURL=False)

# Lê e Ajusta a Tabela

Para edições e limpeza da tabela, os dados são lidos, aonde já se define parâmetros para pular linhas iniciais, ignorar as notas de rodapé etc.

In [ ]:
file = os.path.join('data', filename)

%run '../codes/files/predict_encoding.py'

file_encoding = predict_encoding(file)
print(file_encoding)

df = pd.read_csv(file,
                 delimiter=';',
                 header=1,
                 skiprows=range(1, 3),
                 nrows=5570,
                 encoding=file_encoding)

In [ ]:
# Ajusta o Índice
df.rename(columns={'Cód.': 'ID_Município'}, inplace=True)
df.rename(columns={'Município': 'Município_Orig'}, inplace=True)

# Set Index
df = df.set_index('ID_Município')

# Cria/desmembra colunas
df['Município'] = df['Município_Orig'].str.split(' \(').str[0]
df['Estado'] = df['Município_Orig'].str.split(' \(').str[1]
df['Estado'] = df['Estado'].replace('\)', '', regex=True)

# Exclui coluna
df = df.drop(['Município_Orig'], axis=1)

# Ajusta a ordem das colunas
col = df.columns.to_list()
col.insert(0, col.pop(col.index('Município')))
col.insert(1, col.pop(col.index('Estado')))
df = df.reindex(columns=col)

df

In [ ]:
# Lista de Colunas a serem alteradas
list_columns = df.columns[2:]
list_columns

In [ ]:
# Função que Altera
for i in list_columns:
    year = str(i)
    df[year] = pd.to_numeric(df[year], errors='coerce', downcast='float') 

In [ ]:
# Resultados
print(df.dtypes)
df.head(5)

# Escreve a Tabela

In [ ]:
def write_results(df, filename):
    # Escreve em arquivo CSV    
    df.to_csv(filename,
              encoding='utf-8-sig',
              index=True,
              header=True,
              sep=';',
              decimal=',')
    
    return print('Tabela Salva!')

In [ ]:
file = os.path.join('data', 'tab_EstimativaPopulação_Ajusted.csv')

write_results(df, file)

# Atualiza Repositório no *GitHub*

In [ ]:
%run '../codes/git/update_github.py'

git_full('/home/michel/Geodata/SourceCode/data_population', '.', 'Atualizando')